In [1]:
from datasets import load_dataset
from datasets import Dataset

In [2]:
from sklearn.model_selection import train_test_split
import sys 
sys.path.append('..')
from data.loadData import MovieReviews
valid_size=0.3 # validation + testing
test_size=0.5 # tesing / (testing+validation)
seed=42

df = MovieReviews()
train_data, temp_data = train_test_split(df, test_size=valid_size, random_state=seed)
val_data, test_data = train_test_split(temp_data, test_size=test_size, random_state=seed)

In [25]:
from datasets import DatasetDict
trainDataset = Dataset.from_pandas(train_data)
testDataset = Dataset.from_pandas(test_data)
valDataset = Dataset.from_pandas(val_data)

dataset_dict = DatasetDict({
    "train": trainDataset,
    "test": testDataset,
    "validation": valDataset
})

In [27]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

tokenized_data = dataset_dict.map(preprocess_function, batched=True)

In [29]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [30]:
import evaluate
accuracy = evaluate.load("accuracy")

In [32]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [33]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
# test_dataset = Dataset.from_pandas(val_data)
# tokenized_test_dataset = preprocess_function(test_dataset, batched=True)
test_results = trainer.predict(tokenized_data['test'])

In [41]:
test_results.metrics

{'test_loss': 0.36864280700683594,
 'test_accuracy': 0.8425,
 'test_runtime': 3.8157,
 'test_samples_per_second': 419.323,
 'test_steps_per_second': 26.208}